In [40]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('/Users/addisonpelayo/Desktop/reviews_Automotive_5.json.gz')

In [41]:
df.head(10)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A3F73SC1LY51OO,B00002243X,Alan Montgomery,"[4, 4]",I needed a set of jumper cables for my new car...,5.0,Work Well - Should Have Bought Longer Ones,1313539200,"08 17, 2011"
1,A20S66SKYXULG2,B00002243X,alphonse,"[1, 1]","These long cables work fine for my truck, but ...",4.0,Okay long cables,1315094400,"09 4, 2011"
2,A2I8LFSN2IS5EO,B00002243X,Chris,"[0, 0]",Can't comment much on these since they have no...,5.0,Looks and feels heavy Duty,1374710400,"07 25, 2013"
3,A3GT2EWQSO45ZG,B00002243X,DeusEx,"[19, 19]",I absolutley love Amazon!!! For the price of ...,5.0,Excellent choice for Jumper Cables!!!,1292889600,"12 21, 2010"
4,A3ESWJPAVRPWB4,B00002243X,E. Hernandez,"[0, 0]",I purchased the 12' feet long cable set and th...,5.0,"Excellent, High Quality Starter Cables",1341360000,"07 4, 2012"
5,A1ORODEBRN64C,B00002243X,"James F. Magowan ""Jimmy Mac""","[1, 1]","These Jumper cables are heavy Duty, Yet easy t...",5.0,Compact and Strong !,1258156800,"11 14, 2009"
6,A2R49ZN3G6FTCQ,B00002243X,John M. Harrell,"[1, 1]",bought these for my k2500 suburban plenty of l...,5.0,nice cables,1326153600,"01 10, 2012"
7,A1Q65KYDKXIX8E,B00002243X,Leeland H.,"[0, 0]",these are good enough to get most motorized ve...,5.0,for cars and pickups,1371081600,"06 13, 2013"
8,A3BI8BKIHESDNQ,B00002243X,L. J. Cunningham,"[0, 0]",The Coleman Cable 08665 12-Feet Heavy-Duty Tru...,4.0,Coleman Cable 08665 12-Feet Heavy-Duty Truck a...,1374105600,"07 18, 2013"
9,A1R089P5AS26UE,B00002243X,Mike,"[0, 0]","I have an old car, Its bound to need these som...",5.0,Beefy,1390348800,"01 22, 2014"


In [42]:
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

# Count the number of times each rating appears in the dataset
data = df['overall'].value_counts().sort_index(ascending=False)

# Create the histogram
trace = go.Bar(x = data.index,
               text = ['{:.1f} %'.format(val) for val in (data.values / df.shape[0] * 100)],
               textposition = 'auto',
               textfont = dict(color = '#000000'),
               y = data.values,
               )
# Create layout
layout = dict(title = 'Distribution Of {} automotive ratings'.format(df.shape[0]),
              xaxis = dict(title = 'Rating'),
              yaxis = dict(title = 'Count'))
# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [43]:
# Number of ratings per auto
data = df.groupby('asin')['overall'].count()

# Create trace
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,size = 2))
# Create layout
layout = go.Layout(title = 'Distribution Of Number of Ratings Per auto',
                   xaxis = dict(title = 'Number of Ratings Per auto'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [44]:
# Number of ratings per user
data = df.groupby('reviewerID')['overall'].count()

# Create trace
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0, size = 2))
# Create layout
layout = go.Layout(title = 'Distribution Of Number of Ratings Per User',
                   xaxis = dict(title = 'Ratings Per User'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [45]:
from surprise import Dataset
from surprise import Reader

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['reviewerID', 'asin', 'overall']], reader)

In [46]:
import numpy as np
from sklearn.model_selection import train_test_split

# df2 = df.iloc[:30000, :]
# df2 = df2.groupby('reviewerID').filter(lambda x: len(x) >= 5)
data_train, data_test = train_test_split(df, test_size=0.20, random_state=42, stratify=df['reviewerID'])

In [47]:
from surprise import Dataset
from surprise import Reader
reader = Reader(rating_scale=(1, 5))
data_train_cf = Dataset.load_from_df(data_train[['reviewerID', 'asin', 'overall']], reader)
data_test_cf = Dataset.load_from_df(data_test[['reviewerID', 'asin', 'overall']], reader)

In [48]:
from surprise import KNNBaseline

# To use item-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between items
}
knn = KNNBaseline(sim_options=sim_options)

In [49]:
from surprise.model_selection import cross_validate

results = cross_validate(knn, data, measures=['RMSE'], cv=3, verbose=True)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBaseline on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9763  0.9644  0.9502  0.9636  0.0106  
Fit time          0.20    0.15    0.16    0.17    0.02    
Test time         0.09    0.09    0.08    0.09    0.00    


In [50]:
from surprise import NormalPredictor

cross_validate(NormalPredictor(), data, measures=['RMSE'], cv=3, verbose=True)

Evaluating RMSE of algorithm NormalPredictor on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.1881  1.2066  1.1901  1.1949  0.0083  
Fit time          0.02    0.02    0.02    0.02    0.00    
Test time         0.05    0.04    0.04    0.04    0.00    


{'test_rmse': array([1.18805307, 1.20661922, 1.19009824]),
 'fit_time': (0.024615764617919922,
  0.022651195526123047,
  0.017836809158325195),
 'test_time': (0.045330047607421875,
  0.03939485549926758,
  0.036788225173950195)}

In [51]:
from surprise import KNNBaseline
from surprise.model_selection import GridSearchCV

sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNBaseline, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matr

In [52]:
from surprise import AlgoBase
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import PredictionImpossible

class GroupAlgorithm(AlgoBase):

    def __init__(self, sim_options={}, bsl_options={}):

        AlgoBase.__init__(self, sim_options=sim_options, bsl_options=bsl_options)

    def fit(self, trainset):

        AlgoBase.fit(self, trainset)

        # Compute baselines and similarities
        self.bu, self.bi = self.compute_baselines()
        self.sim = self.compute_similarities()

        return self

    def estimate(self, u, i):

        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            raise PredictionImpossible('User and/or item is unknown.')

        # Compute similarities between u and v, where v describes all other users that have also rated item i.
        neighbors = [(v, self.sim[u, v]) for (v, r) in self.trainset.ir[i]]
        # Sort these neighbors by similarity
        neighbors = sorted(neighbors, key=lambda x: x[1], reverse=True)

        print('The 3 nearest neighbors of user', str(u), 'are:')
        for v, sim_uv in neighbors[:3]:
            print('user {0:} with sim {1:1.2f}'.format(v, sim_uv))

        # Return the baseline estimate
        bsl = self.trainset.global_mean + self.bu[u] + self.bi[i]
        return bsl


In [53]:
from surprise import SVDpp

# We'll use the famous SVD algorithm.
svd = SVDpp()

results = cross_validate(svd, data, measures=['RMSE'], cv=3, verbose=False)

In [54]:
from surprise import SVD
from surprise import BaselineOnly
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering

benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), BaselineOnly(), CoClustering()]:
    
    print("Testing {}".format(algorithm))
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')    

Testing <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7fb738cc6950>
Testing <surprise.prediction_algorithms.matrix_factorization.SVDpp object at 0x7fb6f08e1d50>
Testing <surprise.prediction_algorithms.slope_one.SlopeOne object at 0x7fb737cba050>
Testing <surprise.prediction_algorithms.matrix_factorization.NMF object at 0x7fb6f4f9dbd0>
Testing <surprise.prediction_algorithms.random_pred.NormalPredictor object at 0x7fb6f0760650>
Testing <surprise.prediction_algorithms.knns.KNNBaseline object at 0x7fb6f0760250>
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Testing <surprise.prediction_algorithms.baseline_only.BaselineOnly object at 0x7fb6f0760990>
Estimating biases using als...
Estimating biases using als

,test_rmse,fit_time,test_time
Algorithm,,,
SVDpp,0.897118,2.080313,0.117149
BaselineOnly,0.899120,0.028435,0.026005
SVD,0.902325,0.758418,0.039811
KNNBaseline,0.985429,0.230960,0.150279
CoClustering,1.024727,0.535664,0.028361
SlopeOne,1.065081,0.106033,0.053912
NMF,1.138580,1.069390,0.035761
NormalPredictor,1.192094,0.018048,0.038095


In [55]:
# Execute KNN
sim_options = {'name': 'pearson_baseline', 'user_based': False}
knn = KNNBaseline(sim_options=sim_options)
knn.fit(data.build_full_trainset())

# Target item to analyze its neighbourhood
auto_name = 'B00002243X'

# Get the closes neighbourds
neighbors = knn.get_neighbors(knn.trainset.to_inner_iid(auto_name), k=10)
# Translate the internal ids used in the algorithm to the movie names
neighbors = (knn.trainset.to_raw_iid(asin) for asin in neighbors)

print()
print('The 10 nearest neighbors of {} are:\n'.format(auto_name))
for asin in neighbors:
    print("\t",asin)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.

The 10 nearest neighbors of B00002243X are:

	 B000P17NXQ
	 B0002KKTT0
	 B0024E6QX0
	 B00002243Z
	 B00008BKX5
	 B00008RW9U
	 B00008RW9V
	 B00009RAX7
	 B00009W3G7
	 B0000ATZD5


In [57]:
auto_bias = [(b, data.build_full_trainset().to_raw_iid(i)) for i, b in enumerate(svd.bi)]
print("Top 15 autos:")
sorted(auto_bias, key=lambda x: x[0])[:15]

Top 15 autos:


[(-1.36542664126123, 'B001GH5PX8'),
 (-1.0837947769817435, 'B004IME2ZK'),
 (-1.0486430916929446, 'B00353PWAW'),
 (-1.0113409915277172, 'B0007RDVFC'),
 (-0.9935521325098646, 'B000ET9SB4'),
 (-0.9926237364380011, 'B001F85OUM'),
 (-0.9633502762170817, 'B0010E00T6'),
 (-0.9350679305836986, 'B003UXFNTW'),
 (-0.8963105471430417, 'B00198GE8Y'),
 (-0.8921893269069259, 'B003UT3S6Q'),
 (-0.8901345517461876, 'B000RGQ4Q6'),
 (-0.8516976632579037, 'B0034KH2IQ'),
 (-0.847917866862177, 'B0006Q16YM'),
 (-0.8457707169051051, 'B0009YDQE4'),
 (-0.8383766538469536, 'B00192JF9U')]

In [58]:
print("Top 15 autos:")
sorted(auto_bias, key=lambda x: -x[0])[:15]

Top 15 autos:


[(0.46045269132649, 'B0000UUX34'),
 (0.38018859277547434, 'B000IZ2CIK'),
 (0.3737995139255748, 'B00407RQCY'),
 (0.36651649780163326, 'B000CO9FF0'),
 (0.36645975440496814, 'B0002SRL0C'),
 (0.3566020648003632, 'B000CNBI1A'),
 (0.34835006587622364, 'B000HBNZJA'),
 (0.3456082287981392, 'B003G0KCCM'),
 (0.3447790276428974, 'B0009IQZQG'),
 (0.34392316212671187, 'B003A8D6YG'),
 (0.34391821510720116, 'B0015Y9358'),
 (0.34278966918978154, 'B000C5O1VW'),
 (0.3420653146667406, 'B0007QGT34'),
 (0.3329558693968668, 'B000DZBP60'),
 (0.3319923257910402, 'B0006IX698')]

In [59]:
user_bias = [(b, data.build_full_trainset().to_raw_uid(i)) for i, b in enumerate(svd.bu)]
sorted(user_bias, key=lambda x: x[0])[-1]

(0.46788313919509456, 'A1F92EK0MUNOVR')

In [60]:
df[df.reviewerID == 'A1F92EK0MUNOVR']

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
2158,A1F92EK0MUNOVR,B00068XCQU,Pecos Bill,"[1, 1]",I doubt this thing needs another 5-star review...,5.0,Another 5-star review,1375315200,"08 1, 2013"
14228,A1F92EK0MUNOVR,B0030B2N10,Pecos Bill,"[0, 0]",Originally I just wanted a compressor but when...,5.0,Very Useful,1369699200,"05 28, 2013"
14894,A1F92EK0MUNOVR,B003DKKN7Y,Pecos Bill,"[0, 0]","I have a Toyota FJ Cruiser, which is a good bi...",5.0,Good extendable brush,1368662400,"05 16, 2013"
19525,A1F92EK0MUNOVR,B00APL77NW,Pecos Bill,"[0, 0]",This thing works just like you'd expect and ha...,5.0,"Works great, good connection options",1378339200,"09 5, 2013"
19809,A1F92EK0MUNOVR,B00B7GC50Y,Pecos Bill,"[0, 0]",I feel like I should dock them a star for usin...,5.0,Maintainer not a charger!,1366156800,"04 17, 2013"


In [61]:
sorted(user_bias, key=lambda x: x[0])[0]

(-1.3418167263157614, 'AS4R3I9KU5JVH')

In [62]:
df[df.reviewerID == 'AS4R3I9KU5JVH']

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
3703,AS4R3I9KU5JVH,B000A09MCW,Dobrin'a,"[0, 0]","Well, it does what it was designed for. I have...",5.0,Works as expected,1398384000,"04 25, 2014"
4502,AS4R3I9KU5JVH,B000BPLNXC,Dobrin'a,"[4, 4]",A well made piece. This brush is not too stiff...,5.0,Proper length. Convenient design.,1254441600,"10 2, 2009"
5351,AS4R3I9KU5JVH,B000C9WJ3A,Dobrin'a,"[0, 0]","Inexpensive, yet works as good as the ones sol...",5.0,Great product for the money,1374969600,"07 28, 2013"
8929,AS4R3I9KU5JVH,B000VKOD9S,Dobrin'a,"[0, 0]",I've been using these aftermarket filters for ...,5.0,Works well,1398384000,"04 25, 2014"
9823,AS4R3I9KU5JVH,B0014Y82UQ,Dobrin'a,"[1, 1]",After 2 months of heavy use these still go str...,5.0,Works Great,1398384000,"04 25, 2014"
18437,AS4R3I9KU5JVH,B007TG7HFO,Dobrin'a,"[1, 1]",I use Michelin wiper for the driver side and B...,5.0,Works well,1398384000,"04 25, 2014"
